# Duplicate Data

A data set might have duplicate data: in other words, the same record is represented multiple times. Sometimes, it's easy to find and eliminate duplicate data like when two records are exactly the same. At other times, like what was discussed in the video, duplicate data is hard to spot. 

# Exercise 1

From the World Bank GDP data, count the number of countries that have had a project totalamt greater than 1 billion dollars (1,000,000,000). To get the count, you'll have to remove duplicate data rows.

In [ ]:
import pandas as pd

# read in the projects data set and do some basic wrangling 
projects = pd.read_csv('../data/projects_data.csv', dtype=str)
projects.drop('Unnamed: 56', axis=1, inplace=True)
projects['totalamt'] = pd.to_numeric(projects['totalamt'].str.replace(',', ''))
projects['countryname'] = projects['countryname'].str.split(';', expand=True)[0]
projects['boardapprovaldate'] = pd.to_datetime(projects['boardapprovaldate'])

# TODO: filter the data frame for projects over 1 billion dollars
# TODO: count the number of unique countries in the results
projects[projects['totalamt'] > 1000000000]['countryname'].nunique()

# Exercise 2 (challenge)

This exercise is more challenging. The projects data set contains data about Yugoslavia, which was an Eastern European country until 1992. Yugoslavia eventually broke up into 7 countries: Bosnia and Herzegovina, Croatia, Kosovo, Macedonia, Montenegro, Serbia, and Slovenia.

But the projects dataset has some ambiguity in how it treats Yugoslavia and the 7 countries that came from Yugoslavia. Your task is to find Yugoslavia projects that are probably represented multiple times in the data set.

In [ ]:
# TODO: output all projects for the 'Socialist Federal Republic of Yugoslavia'
# HINT: You can use the exact country name or use the pandas str.contains() method to search for Yugoslavia
projects[projects['countryname'].str.contains('Yugoslavia')]

Yugoslavia officially ended on [April 27th, 1992](https://en.wikipedia.org/wiki/Yugoslavia). 

In the code cell below, filter for projects with a 'boardapprovaldate' prior to April 27th, 1992 **and** with 'countryname' Bosnia and Herzegovina, Croatia, Kosovo, Macedonia, Serbia **or** Slovenia. You'll see there are a total of 12 projects in the data set that match this criteria. Save the results in the republics variable

In [ ]:
import datetime

# TODO: filter the projects data set for project boardapprovaldate prior to April 27th, 1992 AND with countryname
#  of either 'Bosnia and Herzegovina', 'Croatia', 'Kosovo', 'Macedonia', 'Serbia', or 'Sovenia'. Store the
#  results in the republics variable
republics = projects[(projects['boardapprovaldate'] < datetime.date(1992, 4, 27)) &
         ((projects['countryname'].str.contains('Bosnia')) | 
         (projects['countryname'].str.contains('Croatia')) | 
         (projects['countryname'].str.contains('Kosovo')) | 
         (projects['countryname'].str.contains('Macedonia')) | 
         (projects['countryname'].str.contains('Montenegro')) | 
         (projects['countryname'].str.contains('Serbia')) | 
         (projects['countryname'].str.contains('Slovenia')))][['regionname', 
                                                                   'countryname', 
                                                                   'lendinginstr', 
                                                                   'totalamt', 
                                                                   'boardapprovaldate',
                                                               'location', 
                                                               'GeoLocID', 
                                                               'GeoLocName',
                                                               'Latitude', 
                                                               'Longitude', 
                                                               'Country', 
                                                               'project_name']].sort_values('boardapprovaldate')

republics

Are these projects also represented in the data labeled Yugoslavia? In the code cell below, filter for Yugoslavia projects approved between February 1st, 1980 and May 23rd, 1989 which are the minimum and maximum dates in the results above. Store the results in the yugoslavia variable.

The goal is to see if there are any projects represented more than once in the data set.

In [ ]:
# Return Yugoslavia projects that might overlap with the other country projects
yugoslavia = projects[(projects['countryname'].str.contains('Yugoslavia')) & 
         (projects['boardapprovaldate'] >= datetime.date(1980, 2, 1)) &
         (projects['boardapprovaldate'] <= datetime.date(1989, 5, 23))][['regionname', 
                                                                   'countryname', 
                                                                   'lendinginstr', 
                                                                   'totalamt', 
                                                                   'boardapprovaldate',
                                                               'location', 
                                                               'GeoLocID', 
                                                               'GeoLocName',
                                                               'Latitude', 
                                                               'Longitude', 
                                                               'Country', 
                                                                        'project_name']].sort_values('boardapprovaldate')
yugoslavia

And as a final step, try to see if there are any projects in the republics variable and yugoslavia variable that could be the same project.

There are multiple ways to do that. As a suggestion, find unique dates in the republics variable. Then separately find unique dates in the yugoslavia variable. Concatenate (ie append) the results together. And then count the number of times each date occurs in this list. If a date occurs twice, that means the same boardapprovaldate appeared in both the Yugoslavia data as well as in the republics data.

You'll should find that there are three suspicious cases:

* July 26th, 1983
* March 31st, 1987
* October 13th, 1987
* May 23rd, 1989

In [ ]:
import numpy as np

# TODO: find the unique dates in the republics variable
republic_unique_dates = republics['boardapprovaldate'].unique()

# TODO: find the unique dates in the yugoslavia variable
yugoslavia_unique_dates = yugoslavia['boardapprovaldate'].unique()

# TODO: make a list of the results appending one list to the other
dates = np.append(republic_unique_dates, yugoslavia_unique_dates)

# TODO: print out the dates that appeared twice in the results
unique_dates, count = np.unique(dates, return_counts=True)

for i in range(len(unique_dates)):
    if count[i] == 2:
        print(unique_dates[i])

# Conclusion

On July 26th, 1983, for example, projects were approved for Bosnia and Herzegovina, Croatia, Macedonia, Slovenia, and Yugoslavia. The code below shows the projects for that date. You'll notice that Yugoslavia had two projects, one of which was called "Power Transmission Project (03) Energy Managem...". The projects in the other countries were all called "POWER TRANS.III". 

This looks like a case of duplicate data. What you end up doing with this knowledge would depend on the context. For example, if you wanted to get a true count for the total number of projects in the data set, should all of these projects be counted as one project? 

Run the code cell below to see the projects in question.

In [ ]:
import datetime

# run this code cell to see the duplicate data
pd.concat([yugoslavia[yugoslavia['boardapprovaldate'] == datetime.date(1983, 7, 26)], republics[republics['boardapprovaldate'] == datetime.date(1983, 7, 26)]])